In [1]:
from IPython.display import Image

import mlflow
import os
import json

import numpy as np
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.linear_model import LinearRegression

In [ ]:
your_mlflow_url = "xxx"
mlflow.set_tracking_uri(your_mlflow_url_mlflow_url)

# A Hitchhiker’s Guide to MLOps

### Don't Panic and Grab Your ~~Towel~~ Notebook!



## Davide Frazzetto | June 2022

# whoami

- day
    - MLOps Engineer @Massive Entertainment | A Ubisoft Studio
- night
    - Guitarist in a completely unknown metal band



<img src="figures/massive.jpg" alt="tay" width="700"/>

# You said MLOps? Do I need to care about it?


### Remember Tay?
<img src="figures/tay.PNG" alt="tay" width="400"/>




Speakers notes: Microsoft BOT tweets collage. released 2016, down in less than 24h


# What are your KPIs?

![racism_police](figures/racism.png)







- If they had a monitoring system tracking as KPI Bot racism, they would have noticed things going wrong before it was too late
- This shows there are two important points:
    - Set your KPI right! Monitoring the wrong things is not really that useful
    - Live monitoring

# Do I Need to Monitor my ML?

### AI philosophy 101

![Free AI?](figures/blade.gif)




*Speaker note: Not all AIs results are as visible and easy to grasp as the Microsoft Bot. Some will simply cost you millions.*

# Question time

<img src="figures/42.PNG" alt="42" width="700"/>



Speakers notes:

- Who has deployed an ML app to production?
- Who has tracked/monitored an ML app?

# Today

- DevOps and MLOps: a panicless introduction
- Tracking with MLFlow
- Monitoring with Grafana
- Helpful tips and resources


- Here at Pydata to answer the question if you need to monitor
- Going to show some practical examples on how to get started if you are working in Python
    - And actually, the tools I will show are generic for many languages and frameworks

# Key principles of DevOps:

- Automation
- Iteration
- Self-service
- Continuous Development/Integration
- Continuous testing
- Collaboration

# DevOps is Good!

<img src="figures/devops.webp" alt="devops" width="800"/>

<img src="figures/devops_vs_mlops.png" alt="devops_mlops" width="800"/>



*Speaker note: Explain multidimensional ML challenge vs software engineering (code VS data), best with picture*

# MLOps: A Panicless Introduction

### What is it?

MLOps provides a set of **standardized processes** and technology capabilities for **building**, **deploying**,
and **operationalizing** ML systems rapidly and reliably.



<img src="figures/cycle_1.png" alt="mlops_lifecycle" width="500"/>

Image credit: [Practitioners Guide to MLOps](https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf)




*Speaker notes*


Continuous Monitoring: Typical assets produced in this process include the following:
- Anomalies detected in serving data during drift detection
- Evaluation metrics produced from
continuous evaluation
Core MLOps capabilities:
- Dataset & feature repository
- Model monitoring
- ML metadata & artifact repository

# What happens after deployment?



## "Ch-Ch-Ch-Ch-Chan Changes."
-- *Davide Bowie*




<img src="figures/change.gif" alt="changes" width="200"/>

Speaker notes: we will focus on monitoring

<img src="figures/monitor_meme.jpg" alt="anakin" width="500"/>




# Part 3 - Tracking with MLFlow


<img src="figures/mlflow_main.png" alt="mlflow_main" width="600"/>
<img src="figures/mlflow_famous.png" alt="mlflow_famous" width="600"/>




Also: More than 10 talks only at Pydata within the last 2 years.



*Speaker note: version 1 came out in 2019, version 2 is work in progress. Idea is to standardize the tools that are needed in developing ML.*

*Speaker note: I am not affiliated with Databricks. MLFlow is now an open source project, and as a MLEngineer you should make sure the project you are going to put in your roadmap is healthy and well maintained.*


<img src="figures/mlflow_arch.jpg" alt="mlflow" width="800"/>

<img src="figures/mlflow.png" alt="mlflow" width="800"/>


Image credit: [databricks.com/it/blog/2020/04/15/databricks-extends-mlflow-model-registry-with-enterprise-features.html](https://databricks.com/it/blog/2020/04/15/databricks-extends-mlflow-model-registry-with-enterprise-features.html)

# Experiments and Runs

<img src="figures/mlflow_ui.gif" alt="mlflow_ui"/>

In [ ]:
# create an experiment
mlflow.set_experiment("pydata_2022")

# start a run
mlflow.start_run()
# ... all the code in here is part of a run!
mlflow.end_run()

# What should I track?

- Data Quality
- Model Performance
- Code, Configurations, parameters
- Datasets

# Drifting is dangerous!

<img src="figures/drifting.gif" alt="mlflow_ui" width="600"/>

In [ ]:
# Logging some basic data metrics for a run

mlflow.start_run()
# ...
mlflow.log_metric("training_size", 1000)
mlflow.log_metric("validation_size", 500)
mlflow.log_metric("dropped_rows", 34)
mlflow.log_metric("empty_labels", 42)
mlflow.end_run()

In [ ]:
# MOAR METRICS!
metrics = {
    # tracking drifting
    "feature_1_psi": 3.6,
    "feature1_jsd": 0.7,
    "feature1_norm_dist": 4
    
    # OR also pyspark df.summary(), pandas df.describe(), etc.
}

mlflow.start_run()
mlflow.log_metrics(metrics)
mlflow.stop_run()

In [ ]:
# Context managers are nice!

with mlflow.start_run():
    # All your perfect ML code here ...
    # ... which crashes!
    raise IndexError # MLFlow run not closed: FAILED!

    
    # More perfect code ...

In [ ]:
# But you can also log tags!
with mlflow.start_run():
    mlflow.set_tags({
        "training_status": "DONE", 
         "panic_status": "Don't"})

In [ ]:
# Logging datasets or configurations as artifacts!

configuration = {"important_setting": 42, "param_2": "dolphins"}
data = {
    "planet": "Earth", 
    "status": "destroyed",
    "description": "Mostly Harmless"
    }

# Create a local file to save within folder "data"
os.makedirs("data", exist_ok=True)
with open("data/data.json", 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2)


# Write artifacts to root artifact_uri
with mlflow.start_run():
    mlflow.log_artifact("data", "")
    mlflow.log_dict(configuration, "config.json")

# Tracking model quality


- Set your KPIs: Accuracy, Precision, F1score, NDCG, etc.
    - EZ!


- Log them as metrics of a run!
    - We know how to now!

# MLFLow Autlog

Interfaces with the most common ML libraries, eg scikit-learn.

### How does it work?

In sklearn **autologging** is performed when you call:

- estimator.fit()
- estimator.fit_predict()
- estimator.fit_transform()

In [ ]:
# Autolog of sklearn Linear Regression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Enable autologging, automatically recognizes sklearn package
mlflow.autolog()

# Start logging a run
with mlflow.start_run() as run:
    model = LinearRegression()
    model.fit(X, y)



<img src="figures/mlflow_run_details.gif" alt="mlflowrun" width="600"/>

# Part 4 -  Monitoring with Grafana

### Python has great dashboarding options 

<img src="figures/python_visualization.png" alt="akk_the_python_visualizations" width="300"/>

<img src="figures/grafana.png" alt="plugins_1" width="200"/>


- No code required: quick start!
- Runs on virtually any modern platform.
- Loads of plugins for connections and visualizations.


- runs on everything: win, linux, docker, k8s etc
- So unless you are like me even doing presentations with Python, maybe you want to take a look at this!

# Grafana dashboards


<img src="figures/grafana_1.GIF" alt="grafna_1" width="1000"/>

# Remember to setup ALERTS!

<img src="figures/grafana_alert.gif" alt="alert" width="600"/>

# Connection to REST API


<img src="figures/grafana_rest.gif" alt="grafana_rest" width="1000"/>

# Let's Wrap Things up With Tips!

## Do I need to monitor my ML?

## Monitoring your ML System is as important as developing it!

When starting an ML project ask yourself this question: 
**Do you want to release and monitor this project?**
- If no, just hack away in your notebook. But **are you** really **sure**?
- If yes, start with an **MLOps mindset**.
- Monitoring with open source tools is **easy**, and once platform it is setup for one project it is easy to reuse!




*Speaker note: Worked for months on a project. Finally my data is clean, DNN is hyper-tuned, cross-validated, trained and deployed. Done! But was I really?*

## Are you an ML Product Owner?
- Things in ML are moving fast!
- **ML takes time**! Plan in advance and consider your cycle end to end!



<img src="figures/careers.png" alt="careers" width="600"/>

Image credit: [altexsoft.com/blog/machine-learning-engineer/](https://www.altexsoft.com/blog/machine-learning-engineer/)

# Resources

Get started with MLFlow [mlflow.org/docs/latest/quickstart.html](https://mlflow.org/docs/latest/quickstart.html)

Setup a Grafana instance [grafana.com/docs/grafana/next/setup-grafana/installation/](https://grafana.com/docs/grafana/next/setup-grafana/installation/)

Get some Grafana Dashboards templates: [grafana.com/grafana/dashboards/](https://grafana.com/grafana/dashboards/)


## Can I contribute to these projects?

Absolutely yes!

MLFLow: [https://github.com/mlflow/mlflow](https://github.com/mlflow/mlflow)

Grafana: [https://github.com/grafana/grafana](https://github.com/grafana/grafana)


# Questions?


------------------------


# Contact me at

- **GitHub**: https://github.com/dave-frazzetto
- **Linkedin**: https://www.linkedin.com/in/dfrazz/
- **PS handle**: djdave666066
- **Ubiconnect**: dj_davesan

<img src="figures/division.jpg" alt="drawing" width="300"/> 
peace <3